In [5]:
import os, xml.etree.ElementTree as ET

# Load your Signavio BPMN file
bpmn_file = '/Users/e346104/Desktop/data/z3.bpmn'
tree = ET.parse(bpmn_file)
root = tree.getroot()

# Namespace for BPMN (adjust if needed)
ns = {'bpmn': 'http://www.omg.org/spec/BPMN/20100524/MODEL'}

# Prepare new MxClient-style XML
mx_root = ET.Element('Microflow', attrib={'name': 'GeneratedMicroflow'})

# ID tracking
element_id_counter = 1
sequence_id_counter = 1000

# Dictionaries to track BPMN ID -> MX ID
bpmn_to_mx_id = {}

# Helper function to create a Microflow Object
def create_mx_node(mx_type, name):
    global element_id_counter
    node = ET.Element('MicroflowObject', attrib={
        'id': str(element_id_counter),
        'type': mx_type,
        'name': name
    })
    current_id = element_id_counter
    element_id_counter += 1
    return node, current_id

# Helper function to create a Connection between objects
def create_connection(source_id, target_id):
    global sequence_id_counter
    conn = ET.Element('Connection', attrib={
        'id': str(sequence_id_counter),
        'source': str(source_id),
        'target': str(target_id)
    })
    sequence_id_counter += 1
    return conn

# ---- STEP 1: Create nodes for tasks and gateways ----

for task in root.findall('.//bpmn:task', ns):
    name = task.attrib.get('name', 'UnnamedTask')
    mx_node, mx_id = create_mx_node('Action', name)
    mx_root.append(mx_node)
    bpmn_to_mx_id[task.attrib['id']] = mx_id

for gateway in root.findall('.//bpmn:exclusiveGateway', ns):
    name = gateway.attrib.get('name', 'UnnamedGateway')
    mx_node, mx_id = create_mx_node('ExclusiveSplit', name)
    mx_root.append(mx_node)
    bpmn_to_mx_id[gateway.attrib['id']] = mx_id

for event in root.findall('.//bpmn:startEvent', ns):
    name = event.attrib.get('name', 'StartEvent')
    mx_node, mx_id = create_mx_node('StartEvent', name)
    mx_root.append(mx_node)
    bpmn_to_mx_id[event.attrib['id']] = mx_id

for event in root.findall('.//bpmn:endEvent', ns):
    name = event.attrib.get('name', 'EndEvent')
    mx_node, mx_id = create_mx_node('EndEvent', name)
    mx_root.append(mx_node)
    bpmn_to_mx_id[event.attrib['id']] = mx_id

# ---- STEP 2: Create connections based on sequenceFlows ----

for seq_flow in root.findall('.//bpmn:sequenceFlow', ns):
    source_ref = seq_flow.attrib['sourceRef']
    target_ref = seq_flow.attrib['targetRef']
    if source_ref in bpmn_to_mx_id and target_ref in bpmn_to_mx_id:
        conn = create_connection(bpmn_to_mx_id[source_ref], bpmn_to_mx_id[target_ref])
        mx_root.append(conn)
    else:
        print(f"Warning: Missing mapping for sequence flow {seq_flow.attrib.get('id')}")

# ---- STEP 3: Output the final XML ----

# Output file
output_file = os.path.expanduser('~/data/workflows/out.xml')
mx_tree = ET.ElementTree(mx_root)
mx_tree.write(output_file, encoding='utf-8', xml_declaration=True)

print(f"✅ Conversion complete! Output saved to '{output_file}'")


✅ Conversion complete! Output saved to '/Users/e346104/data/workflows/out.xml'
